In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from starfish import FieldOfView, data
from starfish.types import Axes
from skimage import io

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load Data into Starfish from the Cloud

Still struggling with this cell... attempted to follow instructions at https://github.com/spacetx/starfish/pull/1425. Also, apparently `experiment["fov_000"]` has no attribute `get_images`.

In [13]:
# # Download the osmFISH data
# exp = data.osmFISH(use_test_data=False)

# # Download the spots data as an ImageStack object
# fov = experiment.fov()
# primary_image = exp['fov_000'].get_image('primary')



# # Display the shape of the ImageStack
# # The image data are stored in an x array
# primary_image.xarray.shape


experiment = data.osmFISH(use_test_data=False)
imgs = experiment["fov_000"].get_image(FieldOfView.PRIMARY_IMAGES)

ChecksumValidationError: calculated checksum (bb76dcd56ec715eefcc8ca7bbd3bbb7845048cb4334e306996cd2c2c39548d74) does not match expected checksum (b2486d3109069beb9de625b9edce7dae8fa0f01378e6976f0da30a8cb8b8ba7d)

In [ ]:
primary_image = imgs

# Filter Data

In [3]:
from starfish.image import Filter

filter_ghp = Filter.GaussianHighPass(sigma=(1,8,8), is_volume=True)
imgs_ghp = filter_ghp.run(primary_image, in_place=False)

1it [00:00, 17.59it/s]


In [4]:
from starfish.types import Clip
filter_laplace = Filter.Laplace(sigma=(0.2, 0.5, 0.5), is_volume=True, clip_method=Clip.SCALE_BY_CHUNK)
imgs_ghp_laplace = filter_laplace.run(imgs_ghp, in_place=False)

1it [00:00, 18.74it/s]


# Save Images

In [5]:
mp = imgs_ghp_laplace.max_proj(Axes.ZPLANE)

100%|██████████| 1/1 [00:00<00:00, 64.23it/s]


In [6]:
# %gui qt5
# from starfish import display
# display(mp)

In [7]:
# display(imgs_ghp)

In [8]:
# Get the number of channels in the ImageStack
num_ch = mp.num_chs

# Get the number of round in the ImageStack
num_rnd = mp.num_rounds

# Get the number of z slices in the ImageStack
num_z = mp.num_zplanes

print('%d rounds, %d channels, and %d z slice(s)' % (num_rnd, num_ch, num_z))

1 rounds, 1 channels, and 1 z slice(s)


In [9]:
if not os.path.exists('images'):
    os.mkdir('images')
# Iterate through the images and save them to disk
for rnd in range(num_rnd):
    for channel in range(num_ch):
        for z in range(num_z):
            # Create the file name
            file_name = 'ISS_rnd%d_ch%d_z%d.png' % (rnd, channel, z)
            file_path = './images/%s' % file_name
            
            # Create the selector. This dictionary selects which slice (i.e., round, channel, z coords)
            # to select from the image stack
            selector = {
                Axes.ROUND: rnd,
                Axes.CH: channel,
                Axes.ZPLANE: z
            }
            
            # Get the slice
            (im, _) = mp.get_slice(selector)
            
            # Save the slice
            io.imsave(file_path, im, check_contrast=False)


/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: ./images/ISS_rnd0_ch0_z0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
